In [2]:
#importing the libraries
import pandas as pd
import numpy as np

In [3]:
#read all the csv files
files1=pd.read_csv('Consulting 1.csv',encoding = "ISO-8859-1")
files2=pd.read_csv('Consulting 2.csv',encoding = "ISO-8859-1")
files3=pd.read_csv('Consulting 3.csv')
files4=pd.read_csv('Consulting 4.csv',encoding = "ISO-8859-1")
files5=pd.read_csv('Consulting 5.csv')
files6=pd.read_csv('Consulting 6.csv')
files7=pd.read_csv('Consulting 7.csv')

In [4]:
#select only the rows in files1 who had completed application
files1= files1.loc[files1['completedapplication']==1]
files1.shape

(3059, 59)

In [5]:
#merge all the files in 1 on the basis of candidateid
result0 = pd.merge(files1,files6, on='candidateid',how='left')
result00 = pd.merge(result0,files7, on='jobcandidate',how='left')
result = pd.merge(result00,files5, on='candidateid',how='left')
# result is the final merged file
#result2=pd.merge(files1,files4, on='candidateid', how='left')
files2=pd.merge(files1,files2, on='candidateid', how='left')

In [6]:
#delete the columns from files3
files3= files3.drop('createDate',axis=1)
files3= files3.drop('createBy',axis=1)
files3= files3.drop('modifyDate',axis=1)
files3= files3.drop('modifyBy',axis=1)
files3= files3.drop('id',axis=1)
files3= files3.drop('score1',axis=1)
files3= files3.drop('score2',axis=1)
files3= files3.drop('score3',axis=1)
files3= files3.drop('score4',axis=1)
files3= files3.drop('Unnamed: 0',axis=1)
# rename the column userid with candidateid
files3= files3.rename(columns={'userid':'candidateid'})

In [7]:
#Transforming the files3 dataset
df1=files3.pivot_table(index='candidateid', columns='dimensiontype', values='finalscore')
df1new = pd.DataFrame(df1.to_records())
#merging new dataframe with the result file
df2=pd.merge(result,df1new,on='candidateid',how='left')
#Creating dummies for each applicant source
df2new=pd.get_dummies(df2['source'])
#updating the dataframe with the new one
df2=pd.concat([df2, df2new], axis =1)

In [8]:
#Droping the columns from df2 we don't need
df2=df2.drop('Unnamed: 0', axis=1)
df2=df2.drop('candidateid', axis=1)
df2=df2.drop('jobstatus', axis=1)
df2=df2.drop('source', axis=1)
df2=df2.drop('Applicant_type', axis=1)
df2=df2.drop('AllApplicants', axis=1)
df2=df2.drop('NewApplicant', axis=1)
df2=df2.drop('AccountCreated', axis=1)
df2=df2.drop('SQCompleted', axis=1)
df2=df2.drop('ScreeningPass', axis=1)
df2=df2.drop('completedapplication', axis=1)
df2=df2.drop('newaddress', axis=1)
df2=df2.drop('targetIndustry', axis=1)
df2=df2.drop('targetFunction', axis=1)
df2=df2.drop('assessments_taken', axis=1)
df2=df2.drop('total_assessments', axis=1)
df2=df2.drop('ass_compl_rate', axis=1)
df2=df2.drop('newjoblocation', axis=1)
df2=df2.drop('domain', axis=1)
df2=df2.drop('jobRegion', axis=1)
df2=df2.drop('industry', axis=1)
df2=df2.drop('companyname', axis=1)
df2=df2.drop('employeeCount', axis=1)
df2=df2.drop('Companyage', axis=1)
df2=df2.drop('ShortQScore', axis=1)
df2=df2.drop('jobOpeningStatus', axis=1)
df2=df2.drop('outreachcall', axis=1)
df2=df2.drop('SQanswered', axis=1)
df2=df2.drop('engagementtime', axis=1)
df2=df2.drop('voice_int_status', axis=1)
df2=df2.drop('voice_mandatory', axis=1)
df2=df2.drop('YearMonthApplied', axis=1)

In [9]:
#Converting the assessments_mandatory column in 1 or 0
df2.ix[df2.assessments_mandatory == 'YES', 'assessments_mandatory'] = 1
df2.ix[df2.assessments_mandatory == 'NO', 'assessments_mandatory'] = 0
#Converting the willingtoRelocate column in 1 or 0
df2.willingtoRelocate=df2.willingtoRelocate.str.replace('Yes','1')
df2.willingtoRelocate=df2.willingtoRelocate.str.replace('No','0')

c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [10]:
#importing the libraries
import re
import string
punch=set(string.punctuation)

In [11]:
#Selecting the column on which we need to cluster the Dataframe
x=df2['degree']

In [12]:
#Function for cleaning the text
def clean(text):
    text=text.lower()
    text=re.sub(r'[^a-zA-Z]'," ",text)
    text="".join([i for i in text if i not in punch])
    return text.split()

In [13]:
#Using Tfidf Vectorizer to Vectorize the target column
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0,
                             min_df=0.0,
                             tokenizer=clean,
                             lowercase=False)
Tfidf_matrix = vectorizer.fit_transform(x.values.astype('U'))

In [14]:
#Clustering using KMeans
from sklearn.cluster import KMeans
model = KMeans(n_clusters=4,
               max_iter=300,
               tol=0.001,
               algorithm='auto')
model.fit(Tfidf_matrix)
labels = model.predict(Tfidf_matrix)
model.cluster_centers_

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [15]:
#Making a different column for Degree using the clustering result
df2['Degree']=labels

In [16]:
#Selecting the column on which we need to cluster the Dataframe
x=df2['currentJobTitle']

In [17]:
#Using Tfidf Vectorizer to Vectorize the target column
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0,
                             min_df=0.0,
                             tokenizer=clean,
                             lowercase=False)
Tfidf_matrix = vectorizer.fit_transform(x.values.astype('U'))

In [18]:
#Clustering using KMeans
from sklearn.cluster import KMeans
model = KMeans(n_clusters=6,
               max_iter=300,
               tol=0.001,
               algorithm='auto')
model.fit(Tfidf_matrix)
labels = model.predict(Tfidf_matrix)
model.cluster_centers_

array([[ 0.0012147 ,  0.00110037,  0.0008806 , ...,  0.00057243,
         0.00038514,  0.0032567 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
#Making a different column for Degree using the clustering results
df2['JobTitle']=labels

In [20]:
#A different column using the ratio for Experience
df2['ratiominexp']=df2['totalExperience']/df2['minimumExp']
df2['ratiomaxexp']=df2['totalExperience']/df2['maximumExp']
#A different column using the ratio for Salary
df2['ratiominsalary']=df2['expectedAnnualSalary']/df2['minAnnualSalary']
df2['ratiomaxsalary']=df2['expectedAnnualSalary']/df2['maxAnnualSalary']

In [21]:
#Dropping the columns from df2 
df2=df2.drop('currentJobTitle', axis=1)
df2=df2.drop('currentEmployer', axis=1)
df2=df2.drop('currentJobStatus', axis=1)
df2=df2.drop('currentAnnualSalary', axis=1)
df2=df2.drop('expectedAnnualSalary', axis=1)
df2=df2.drop('currency', axis=1)
df2=df2.drop('salaryType', axis=1)
df2=df2.drop('totalExperience', axis=1)
df2=df2.drop('minAnnualSalary', axis=1)
df2=df2.drop('maxAnnualSalary', axis=1)
df2=df2.drop('minimumExp', axis=1)
df2=df2.drop('maximumExp', axis=1)
df2=df2.drop('ExlcusionReason', axis=1)
df2=df2.drop('degree', axis=1)
df2=df2.drop('college', axis=1)
df2=df2.drop('grade', axis=1)
df2=df2.drop('applieddate', axis=1)

In [22]:
#Renaming a column in files4 
files4= files4.rename(columns={'jobid':'jobId'})
#Making  a jobcandidate column in files4 
files4['jobcandidate']=files4['jobId'].astype(str)+'-'+files4['candidateid'].astype(str)

In [23]:
#Transforming the files4 dataset
df1=files4.pivot_table(index='jobcandidate', columns='assessmentname', values='assessmentscore')
#Filling the Nan with 0
df1=df1.fillna(0)
df1new = pd.DataFrame(df1.to_records())
#Merging the 2 dataframes
df3=pd.merge(df2,df1new,on='jobcandidate',how='left')

In [24]:
#Making different files for different jobId
file101= df3.loc[df3['jobId']==101]
file164= df3.loc[df3['jobId']==164]
file195= df3.loc[df3['jobId']==195]
file199= df3.loc[df3['jobId']==199]
file234= df3.loc[df3['jobId']==234]
file278= df3.loc[df3['jobId']==278]
file280= df3.loc[df3['jobId']==280]
file294= df3.loc[df3['jobId']==294]

In [25]:
#Columns in the particular file
file164.columns

Index(['jobcandidate', 'jobId', 'Shortlisted', 'assessments_mandatory',
       'LocationMatch', 'OpsExperienceMatch', 'SalaryMatch', 'ExperienceMatch',
       'OpsSalaryMatch', 'shortqpercentage', 'ExcludeScore', 'locationscore',
       'industrymatch', 'functionmatch', 'voiceattempts', 'willingtoRelocate',
       'Adventurous', 'Creative', 'Detailed', 'Empathic', 'Energetic',
       'Idealistic', 'Independent', 'Learning Orientated', 'Loyal',
       'Persuasive', 'Poised', 'Professional', 'Resilient', 'Social',
       'Task orientated', 'Team Focused', 'DIRECT', 'Employer general ref 2',
       'IIMJOBS', 'LINKEDIN', 'LINKEDIN MSG', 'MONSTER', 'MONSTERDB', 'NAUKRI',
       'NAUKRIDB', 'OTHER JOB BOARD 1', 'Other active', 'Other ref 1',
       'REFERRAL', 'SHORTLIST DB', 'SOCIAL OTHERS', 'SOCIAL OTHERS 1',
       'TIMES JOBS DB', 'WHATSAPP 1', 'Degree', 'JobTitle', 'ratiominexp',
       'ratiomaxexp', 'ratiominsalary', 'ratiomaxsalary',
       ' Attention to Detail  Powerpoint Error-s

In [26]:
#Selecting the target and train columns
target=['Shortlisted']
train=[ 'assessments_mandatory',
       'LocationMatch', 'OpsExperienceMatch', 'SalaryMatch', 'ExperienceMatch',
       'OpsSalaryMatch', 'shortqpercentage', 'ExcludeScore', 'locationscore',
       'industrymatch', 'functionmatch', 'voiceattempts', 'willingtoRelocate',
       'Adventurous', 'Creative', 'Detailed', 'Empathic', 'Energetic',
       'Idealistic', 'Independent', 'Learning Orientated', 'Loyal',
       'Persuasive', 'Poised', 'Professional', 'Resilient', 'Social',
       'Task orientated', 'Team Focused', 'DIRECT', 'Employer general ref 2',
       'IIMJOBS', 'LINKEDIN', 'LINKEDIN MSG', 'MONSTER', 'MONSTERDB', 'NAUKRI',
       'NAUKRIDB', 'OTHER JOB BOARD 1', 'Other active', 'Other ref 1',
       'REFERRAL', 'SHORTLIST DB', 'SOCIAL OTHERS', 'SOCIAL OTHERS 1',
       'TIMES JOBS DB', 'WHATSAPP 1', 'Degree', 'JobTitle', 'ratiominexp',
       'ratiomaxexp', 'ratiominsalary', 'ratiomaxsalary',
       ' Attention to Detail  Powerpoint Error-spotting (SL)',
       'Analytical Insights - Taxi Market - Consulting (SL) - Short Version',
       'Analytical Insights - Taxi Market - Consulting V2 (SL)',
       'Analytical Insights - Taxi Market - Consulting v2 (SL)',
       'Assessment Beginning - Please Note (SL)',
       'Assessment End - Thank you (SL)',
       'Assessment End - Thank you - Dasra (SL) ',
       'Business Case: Operational Efficiency - India (SL)',
       'Business Case: Profitability (SL)', 'Cognitive Ability - Medium (SL)',
       'Data Analysis (SL)', 'Excel (Shortlist) - v1',
       'Impact Analyst - Xynteo - Understanding of local markets',
       'Market Sizing - India (SL)', 'Project Management - Consulting (SL)',
       'Project Management Juggle - Consulting (SL)',
       'Project Management  External Stakeholders (SL)',
       'Qualitative Market Research (SL)']

In [27]:
#Making the training and target columns
Y=file278[target]
X=file278[train]
#Filling the missing rows with 0
X = X.fillna(0)

In [28]:
X['willingtoRelocate']=X['willingtoRelocate'].astype(int)
X['assessments_mandatory']=X['assessments_mandatory'].astype(int)


In [29]:
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [34]:
from sklearn.model_selection import train_test_split as tts
#Split the Data in training and testing dataset
X_train, X_test, Y_train, Y_test = tts(X,Y, test_size=0.8, random_state = 56)

In [37]:
seed=7
scoring='accuracy'

In [38]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when 

LR: 0.965152 (0.056712)


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\dis

LDA: 0.930303 (0.087026)
KNN: 0.965152 (0.056712)


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\model_selection\_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\model_selection\_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\model_selection\_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\mode

CART: 0.956818 (0.057100)
NB: 0.824242 (0.127750)


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when 

SVM: 0.965152 (0.056712)


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
from sklearn import svm
clf = svm.LinearSVC()
#fit the train and test data
clf.fit(X_train, Y_train)
print (clf)
#checking the accuracy of model
print(clf.score(X_test, Y_test))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.941935483871


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
clf_A = LogisticRegression(random_state = 42)

clf_A.fit(X_train, Y_train)
print(clf_A.score(X_test, Y_test))

0.948387096774


c:\users\ashwani\anaconda3\envs\neuralnets\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
import xgboost as xgb
clf_C = xgb.XGBClassifier(seed = 120)
clf_C.fit(X_train, Y_train)
print(clf_C.score(X_test, Y_test))

ImportError: No module named 'xgboost'

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
Y_pred = clf.fit(X_train, Y_train).predict(X_test)

cnf_matrix = confusion_matrix(Y_test, Y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=target,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=target, normalize=True,
                      title='Normalized confusion matrix')

plt.show()


In [ ]:
file278['Shortlisted'].sum()